### RUN

In [1]:
!pip install pdfminer.six==20221105 spacy docx2txt==0.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3959 sha256=8eab530e990d0f1d2d8f82187354f956258a6c45c93a3877c6adab7d5379ffd8
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [2]:
import re
import spacy
from pdfminer.high_level import extract_text
import docx2txt
import requests
import string
import logging

In [3]:
import gdown
gdown.download(id="1CDV96X0IEAz2Knu-K5qCrrAAu9Uoy7D4", output='skill-model.zip', quiet=False,fuzzy=True)
#https://drive.google.com/file/d/1CDV96X0IEAz2Knu-K5qCrrAAu9Uoy7D4/view?usp=drive_link
gdown.download(id="1nArFoGLxeO_M9lbt6FEf55VCb6AE-6Y7", output='model2.zip', quiet=False,fuzzy=True)
#https://drive.google.com/file/d/1nArFoGLxeO_M9lbt6FEf55VCb6AE-6Y7/view?usp=sharing
gdown.download(id="145snFZTUqbqsK2nWqZLDGVu6pkbKtioI", output='model3.zip', quiet=False,fuzzy=True)
#https://drive.google.com/file/d/145snFZTUqbqsK2nWqZLDGVu6pkbKtioI/view?usp=sharing
gdown.download(id="1IqmCE13IKPsfba89267VFHKg-_xsNWAK", output='sample.pdf', quiet=False,fuzzy=True)
#https://drive.google.com/file/d/1IqmCE13IKPsfba89267VFHKg-_xsNWAK/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1CDV96X0IEAz2Knu-K5qCrrAAu9Uoy7D4
To: /content/skill-model.zip
100%|██████████| 607M/607M [00:17<00:00, 35.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nArFoGLxeO_M9lbt6FEf55VCb6AE-6Y7
To: /content/model2.zip
100%|██████████| 607M/607M [00:17<00:00, 35.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=145snFZTUqbqsK2nWqZLDGVu6pkbKtioI
To: /content/model3.zip
100%|██████████| 607M/607M [00:17<00:00, 35.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1IqmCE13IKPsfba89267VFHKg-_xsNWAK
To: /content/sample.pdf
100%|██████████| 72.3k/72.3k [00:00<00:00, 18.0MB/s]


'sample.pdf'

In [4]:
!unzip skill-model.zip
!unzip model2.zip
!unzip model3.zip

Archive:  skill-model.zip
  inflating: train1/config.cfg       
  inflating: train1/tok2vec/cfg      
  inflating: train1/vocab/vectors.cfg  
  inflating: train1/vocab/lookups.bin  
  inflating: train1/meta.json        
  inflating: train1/ner/moves        
  inflating: train1/ner/cfg          
  inflating: train1/ner/model        
  inflating: train1/tokenizer        
  inflating: train1/vocab/key2row    
  inflating: train1/vocab/strings.json  
  inflating: train1/tok2vec/model    
  inflating: train1/vocab/vectors    
Archive:  model2.zip
  inflating: train3/tok2vec/cfg      
  inflating: train3/ner/cfg          
  inflating: train3/meta.json        
  inflating: train3/tokenizer        
  inflating: train3/ner/moves        
  inflating: train3/config.cfg       
  inflating: train3/ner/model        
  inflating: train3/vocab/lookups.bin  
  inflating: train3/vocab/vectors.cfg  
  inflating: train3/vocab/key2row    
  inflating: train3/vocab/strings.json  
  inflating: train3/tok2vec

###Load Model

In [5]:
skill_model = spacy.load("/content/train1")
model2 = spacy.load("/content/train3")
model3 = spacy.load("/content/model-best")

###Try

In [6]:
#Please provide the path of the resume
path ="/content/sample.pdf"

In [7]:
def extract_email_from_resume(text):
    email = None
    words = text.split()
    email = []
    for word in words:
        if "@" in word:
            email.append(word.strip())
    return email

In [8]:
def extract_contact_number_from_resume(text):
    contact_number = None
    pattern = r"\b(?:\+?\d{1,3}[-.\s]?)?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"
    match = re.search(pattern, text)
    if match:
        contact_number = match.group()

    return contact_number

In [9]:
def extract_link(text):
    link = []
    pattern = r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})"
    links = re.findall(pattern, text)
    return links

In [10]:
def extract_text_from(file_path, extension):
    text = ''
    if extension == '.pdf':
      with open(file_path, 'rb') as f:
        text = extract_text(f)
    elif extension == '.docx' or extension == '.doc':
      with open(file_path, 'rb') as f:
        text = docx2txt.process(file_path)
    elif extension ==".txt":
        text = open(file_path).read()
    return text

In [11]:
extracted_details = {}

In [12]:
contains_punctuation = lambda s: any(char in string.punctuation for char in s)

def extract_deatils(text):
    email = extract_email_from_resume(text)
    extracted_details["email"] = email
    print("Email -", email)
    contact_number = extract_contact_number_from_resume(text)
    print("Contact Number - ",contact_number)
    extracted_details["contact_number"] = contact_number
    links = extract_link(text)
    print("Links -",links)
    extracted_details["links"] = links


    skills = set()
    for ent in skill_model(text).ents:
        if ent.label_ == "HARD_SKILLS":
          if len(ent.text) < 3 or ent.text.isnumeric() or contains_punctuation(ent.text):
            continue
          skills.add(ent.text)

    print("skills -",skills)
    extracted_details["skills"] = skills

In [13]:
dic = {}
features = set()

In [14]:
text = extract_text_from(path ,"."+path.split(".")[-1])
# print(text)
extract_deatils(text)

Email - ['proffapt@pm.me']
Contact Number -  None
Links - []
skills - {'Vim', 'Python', 'MongoDB', 'API', 'fetch the', 'GitHub', 'Java', 'derlying', 'CSS', 'Linux', 'Red', 'Golang', 'Web', 'log management', 'Python\n', 'fERP', 'script', 'Linux System', 'Unix', 'Postgresql', 'debugging', 'HTML', 'Data Structures', 'LaTeX', 'PostgreSQL', 'Git', 'OS Platforms'}


In [15]:
for ent in model2(text).ents:
    if dic.get(ent.label_+"->"+ent.text) != None:
        dic[ent.label_+"->"+ent.text] += 1
    else:
        dic[ent.label_+"->"+ent.text] = 1
    print(ent.label_," --> ", ent.text)
    print("----------------------------------")
    features.add(ent.text)

COLLEGE  -->  Arpit Bhardwaj
----------------------------------
LOCATION  -->  Mathura
----------------------------------
DEGREE  -->  B.Tech. in Industrial and Systems Engineering
----------------------------------
COLLEGE  -->  Indian Institute of Technology
----------------------------------
LOCATION  -->  Kharagpur
----------------------------------
SKILLS  -->  CSS
----------------------------------
COLLEGE  -->  GSYNC
----------------------------------
SKILLS  -->  BASH
----------------------------------
SKILLS  -->  GitHub
----------------------------------
SKILLS  -->  nix
----------------------------------
SKILLS  -->  Linux
----------------------------------
SKILLS  -->  Python
----------------------------------
SKILLS  -->  Java
----------------------------------
SKILLS  -->  JavaScript
----------------------------------
SKILLS  -->  Python
----------------------------------
SKILLS  -->  HTML
----------------------------------
SKILLS  -->  CSS
-------------------------------

In [16]:
for ent in model3(text).ents:
    if dic.get(ent.label_+"->"+ent.text) != None:
        dic[ent.label_+"->"+ent.text] += 1
    else:
        dic[ent.label_+"->"+ent.text] = 1
    print(ent.label_," --> ", ent.text)
    print("----------------------------------")
    features.add(ent.text)

NAME  -->  � proffapt.hashnode.dev
----------------------------------
COLLEGE NAME  -->  Arpit Bhardwaj
----------------------------------
LOCATION  -->  Mathura
----------------------------------
COLLEGE NAME  -->  Indian Institute of Technology, Kharagpur
----------------------------------
EMAIL ADDRESS  -->  fERP
----------------------------------
SKILLS  -->  Languages Bash, Golang, Java, JavaScript, Python, HTML, CSS, lua, C, C++, LaTeX

Data Storage / Handling

PostgreSQL, MongoDB

OS Platforms Nix OS, Unix(Free BSD, Open BSD, Solaris-8,9,10 and MacOS X), Linux 5/6 (Debian, Fedora, Cen-

tOS, Arch, Gentoo and their derivatives), Windows (7, 8, 8.1 and 10)

Others Git and GitHub, Linux System Administration, BackEnd Development, Web Browser Extension
Development, Object Oriented Programming, Penetration-testing (Red Team and Blue Team),
Gurobi & CPLEX Optimisation.

Activities & Leadership
• Co-Founder, Cybernity: I established a cybersecurity community to promote its genuine prin

In [17]:
for keys in dic.keys():
    if keys.split("->")[0] in ["SKILLS","YEARS"] or dic[keys] <= 1:
        continue
    print(keys.split("->")[0]," --> ",keys.split("->")[1])
    if keys.split("->")[0] == "LOCATION":
        if extracted_details.get("LOCATION") == None:
            extracted_details["LOCATION"] = [keys.split("->")[1]]
        else:
            extracted_details["LOCATION"].append(keys.split("->")[1])
    else :
        extracted_details[keys.split("->")[0]] = keys.split("->")[1]

LOCATION  -->  Mathura


In [18]:
last_name = requests.get("https://gist.githubusercontent.com/dudegladiator/af474064777f93105cbd5fc3b025c2eb/raw/9e9cfa7148ee411680f4b8eb6f672fd00d82474b/Indian_Last_names.txt").text.lower().split("\n")
colleges1 =  requests.get("https://gist.githubusercontent.com/dudegladiator/c0e24cd3e2ee8e9ff9ffea2db556a232/raw/7a4ee3083fc4647087bc47d4b26811a482f03926/Indian_colleges.txt").text.lower().split("\n")

In [19]:
colleges = []
for college in colleges1:
    colleges.append(college.strip(" "))
colleges.append("indian institute of technology kharagpur")

In [20]:
for entity in features:
    if str(entity).lower().split()[-1] in last_name:
        extracted_details["name"] = str(entity)
        print(entity)
    if str(entity).lower().replace(",", "") in colleges:
        extracted_details["college"] = str(entity)
        print(entity)

Arpit Bhardwaj
Indian Institute of Technology, Kharagpur


In [21]:
extracted_details

{'email': ['proffapt@pm.me'],
 'contact_number': None,
 'links': [],
 'skills': {'API',
  'CSS',
  'Data Structures',
  'Git',
  'GitHub',
  'Golang',
  'HTML',
  'Java',
  'LaTeX',
  'Linux',
  'Linux System',
  'MongoDB',
  'OS Platforms',
  'PostgreSQL',
  'Postgresql',
  'Python',
  'Python\n',
  'Red',
  'Unix',
  'Vim',
  'Web',
  'debugging',
  'derlying',
  'fERP',
  'fetch the',
  'log management',
  'script'},
 'LOCATION': ['Mathura'],
 'name': 'Arpit Bhardwaj',
 'college': 'Indian Institute of Technology, Kharagpur'}